# Mnist Module

In [1]:
import numpy as np
import onnx
import os
import glob
from onnx import numpy_helper
import tvm
from tvm import te
import tvm.relay as relay

## Load module and param

In [2]:
mnist_model = onnx.load('../mnist/mnist-8.onnx')

In [3]:
input_name = "Input3"
shape_dict = {input_name: (1, 1, 28, 28)}
print(shape_dict)

{'Input3': (1, 1, 28, 28)}


In [4]:
mod, params = relay.frontend.from_onnx(mnist_model, shape_dict)

In [5]:
print(mod.astext(show_meta_data=False))

#[version = "0.0.5"]
def @main(%Input3: Tensor[(1, 1, 28, 28), float32], %Parameter5: Tensor[(8, 1, 5, 5), float32], %Parameter6: Tensor[(8, 1, 1), float32], %Parameter87: Tensor[(16, 8, 5, 5), float32], %Parameter88: Tensor[(16, 1, 1), float32], %Pooling160_Output_0_reshape0_shape: Tensor[(2), int64], %Parameter193: Tensor[(16, 4, 4, 10), float32], %Parameter193_reshape1_shape: Tensor[(2), int64], %Parameter194: Tensor[(1, 10), float32]) {
  %0 = nn.pad(%Input3, 0f, pad_width=[[0, 0], [0, 0], [2, 2], [2, 2]]);
  %1 = nn.conv2d(%0, %Parameter5, padding=[0, 0, 0, 0], kernel_size=[5, 5]);
  %2 = add(%1, %Parameter6);
  %3 = nn.relu(%2);
  %4 = nn.max_pool2d(%3, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %5 = nn.pad(%4, 0f, pad_width=[[0, 0], [0, 0], [2, 2], [2, 2]]);
  %6 = nn.conv2d(%5, %Parameter87, padding=[0, 0, 0, 0], kernel_size=[5, 5]);
  %7 = add(%6, %Parameter88);
  %8 = nn.relu(%7);
  %9 = nn.max_pool2d(%8, pool_size=[3, 3], strides=[3, 3], padding=[0, 0, 0, 0])

## Set Target 

In [6]:
# target_type = 'x86_64'
# target_type = 'ARMv8'
target_type = 'OpenCL'

In [7]:
if target_type == 'x86_64':
    target = tvm.target.Target('llvm')
    target_host = tvm.target.Target('llvm')
elif target_type == 'ARMv7':
    target = tvm.target.arm_cpu("rasp3b")
    target_host = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
elif target_type == 'ARMv8':
    target = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
    target_host = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
elif target_type == 'OpenCL':
    target = tvm.target.Target("opencl -device=intel_graphics")
    target_host = tvm.target.Target("llvm -device=arm_cpu -mtriple=aarch64-linux-gnu -mattr=+neon")
    
print(target)
print(target_host)

opencl -keys=intel_graphics,opencl,gpu -device=intel_graphics -max_num_threads=256 -thread_warp_size=1
llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mattr=+neon -mtriple=aarch64-linux-gnu


## Build

In [8]:
with tvm.transform.PassContext(opt_level=3):
# with relay.build_config(opt_level=3):    
    graph, module_lib, param = relay.build(mod, target=target, target_host=target_host, params=params, mod_name='mnist')

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
/tmp/ipykernel_554798/131324857.py:3: DeprecationWarning: legacy graph executor behavior of producing json / lib / params will be removed in the next release. Please see documents of tvm.contrib.graph_executor.GraphModule for the  new recommended usage.
  graph, module_lib, param = relay.build(mod, target=target, target_host=target_host, params=params, mod_name='mnist')


In [9]:
print(module_lib)

Module(llvm, 2901418)


In [10]:
print(graph)

{
  "nodes": [
    {
      "op": "null", 
      "name": "Input3", 
      "inputs": []
    }, 
    {
      "op": "tvm_op", 
      "name": "tvmgen_mnist_fused_nn_pad_1", 
      "attrs": {
        "num_outputs": "1", 
        "num_inputs": "1", 
        "flatten_data": "0", 
        "func_name": "tvmgen_mnist_fused_nn_pad_1", 
        "hash": "45c511efac18ad00"
      }, 
      "inputs": [
        [
          0, 
          0, 
          0
        ]
      ]
    }, 
    {
      "op": "null", 
      "name": "p0", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "p1", 
      "inputs": []
    }, 
    {
      "op": "tvm_op", 
      "name": "tvmgen_mnist_fused_nn_conv2d_add_nn_relu_1", 
      "attrs": {
        "num_outputs": "1", 
        "num_inputs": "3", 
        "flatten_data": "0", 
        "func_name": "tvmgen_mnist_fused_nn_conv2d_add_nn_relu_1", 
        "out_layout": "", 
        "kernel_layout": "OIHW", 
        "data_layout": "NCHW", 
        "hash": "4866eef6b04ad

In [11]:
print(param)

{'p1': <tvm.nd.NDArray shape=(1, 8, 1, 1), cpu(0)>
array([[[[-0.16153972]],

        [[-0.43383566]],

        [[ 0.09164136]],

        [[-0.01685222]],

        [[-0.06502644]],

        [[-0.13173787]],

        [[ 0.02041755]],

        [[-0.12111023]]]], dtype=float32), 'p0': <tvm.nd.NDArray shape=(8, 1, 5, 5), cpu(0)>
array([[[[-0.00890567, -0.23690744, -0.50882167, -0.06456178,
           0.14181185],
         [-0.59197617, -0.47528538, -0.04934815,  0.7682156 ,
           0.2634652 ],
         [-0.49176344,  0.05561765,  1.0189645 ,  0.5547042 ,
          -0.4416644 ],
         [-0.15953699,  0.5575415 ,  0.59209126, -0.2947414 ,
          -0.6131798 ],
         [ 0.03849885,  0.22601931, -0.2185563 , -0.4771943 ,
          -0.29170495]]],


       [[[-0.03988235,  0.21859981,  0.49872705,  0.42367178,
           0.04854779],
         [-0.10118173, -0.27690947, -0.00439903,  0.5271722 ,
           0.40340957],
         [-0.1508379 , -0.32669756, -0.15898573,  0.35899585,
      

In [12]:
print(module_lib.get_source())

; ModuleID = 'TVMMod'
source_filename = "TVMMod"
target datalayout = "e-m:e-i8:8:32-i16:16:32-i64:64-i128:128-n32:64-S128"
target triple = "aarch64-linux-gnu"

%0 = type { double }
%1 = type { i8*, %2, i32, %3, i64*, i64*, i64 }
%2 = type { i32, i32 }
%3 = type { i8, i8, i16 }

@__tvm_module_ctx = linkonce dllexport local_unnamed_addr global i8* null, align 8
@__TVMFuncCall = linkonce dllexport local_unnamed_addr global i32 (i8*, %0*, i32*, i32, %0*, i32*)* null, align 8
@__TVMBackendGetFuncFromEnv = linkonce dllexport local_unnamed_addr global i32 (i8*, i8*, i8**)* null, align 8
@__TVMAPISetLastError = linkonce dllexport local_unnamed_addr global void (i8*)* null, align 8
@.str = private constant [85 x i8] c"Assert fail: (num_args == 2), tvmgen_mnist_fused_nn_max_pool2d: num_args should be 2\00", align 1
@.str.1 = private constant [160 x i8] c"Assert fail: ((((arg0.code == 3) || (arg0.code == 13)) || (arg0.code == 7)) || (arg0.code == 4)), tvmgen_mnist_fused_nn_max_pool2d: Expect arg[

## Export Lib

In [16]:
# lib export

In [17]:
if target_type == 'x86_64':
    lib_path   = "../module/x86_64/mnist_x86_64.so"
    param_path = "../module/x86_64/mnist_x86_64.params"
    module_lib.export_library(lib_path)
elif target_type == 'ARMv7':
    lib_path   = "../module/armv7/mnist_aarch64.so"
    param_path = "../module/armv7/mnist_aarch64.params"
    module_lib.export_library(lib_path, tvm.contrib.cc.cross_compiler('arm-linux-gnueabihf-g++'))
elif target_type == 'ARMv8':
    lib_path   = "../module/armv8/mnist_aarch64.so"
    param_path = "../module/armv8/mnist_aarch64.params"
    module_lib.export_library(lib_path, tvm.contrib.cc.cross_compiler('aarch64-linux-gnu-g++'))
elif target_type == 'OpenCL':
    lib_path   = "../module/opencl/mnist_opencl.so"
    param_path = "../module/opencl/mnist_opencl.params"
    module_lib.export_library(lib_path, tvm.contrib.cc.cross_compiler('aarch64-linux-gnu-g++'))    

##  Export Params

In [18]:
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))

In [20]:
#module = tvm.contrib.graph_executor.GraphModule(module_lib["mnist"](ctx))